In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import dask.dataframe as dd
import os
import sys
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
import time
from collections import Counter
tqdm.pandas(tqdm_notebook)
import lightgbm as lgb
import gc
import datetime
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
pd.set_option('display.float_format', '{:.5f}'.format)
SEED = 51
np.random.seed(SEED)
%matplotlib inline

In [2]:
FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0','outliers']

In [3]:
%%time
train_df = dd.read_csv('../input/train_3_961.csv')
test_df = dd.read_csv('../input/test_3_961.csv')
train_df = train_df.compute()
test_df = test_df.compute()
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

CPU times: user 12.9 s, sys: 1.72 s, total: 14.6 s
Wall time: 5.7 s


In [4]:
used_features = list(train_df.columns)

In [5]:
print(train_df.shape,test_df.shape)

(201917, 203) (123623, 202)


In [6]:
%%time
train_last = dd.read_csv('../input/train_last.csv')
test_last = dd.read_csv('../input/test_last.csv')
train_last = train_last.compute()
test_last = test_last.compute()
train_last = train_last.reset_index(drop=True)
test_last = test_last.reset_index(drop=True)

CPU times: user 18.1 s, sys: 2.3 s, total: 20.4 s
Wall time: 7.33 s


In [7]:
print(train_last.shape,test_last.shape)

(201917, 287) (123623, 286)


In [8]:
train_df.head()

,card_id,feature_1,feature_2,feature_3,first_active_month,outliers,target,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_month_nunique,hist_month_mean,hist_month_min,hist_month_max,hist_hour_nunique,hist_hour_mean,hist_hour_min,hist_hour_max,hist_weekofyear_nunique,hist_weekofyear_mean,hist_weekofyear_min,hist_weekofyear_max,hist_weekday_mean,hist_day_nunique,hist_day_mean,hist_day_min,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_skew,hist_installments_sum,hist_installments_max,hist_installments_mean,hist_installments_var,hist_installments_skew,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_skew,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_authorized_flag_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_mean,hist_category_3_mean,hist_card_id_size,hist_card_id_count,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_month_mean,new_month_min,new_month_max,new_hour_nunique,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_weekday_mean,new_weekday_min,new_weekday_max,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_skew,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_installments_skew,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_month_lag_var,new_month_lag_skew,new_month_diff_mean,new_month_diff_var,new_month_diff_skew,new_weekend_mean,new_category_1_mean,new_category_2_mean,new_category_3_mean,new_card_id_size,new_card_id_count,new_price_mean,new_price_max,new_price_min,new_price_var,new_Christmas_Day_2017_mean,new_Children_day_2017_mean,new_Black_Friday_2017_mean,new_Mothers_Day_2018_mean,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,new_purchase_date_uptomin,hist_first_buy,hist_last_buy,new_first_buy,new_last_buy,card_id_total,card_id_cnt_total,card_id_cnt_ratio,purchase_amount_total,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_ratio,month_diff_mean,month_diff_ratio,month_lag_mean,month_lag_max,month_lag_min,category_1_mean,installments_total,installments_mean,installments_max,installments_ratio,price_total,price_mean,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio
0,C_ID_92a2005557,0.01314,0.00875,0.01143,2017-06-01,0.000

In [9]:
used_columns = ['card_id'] + [x for x in train_last.columns if x not in train_df.columns]

In [10]:
train = train_df.merge(train_last[used_columns],how = 'left',on = 'card_id')

In [11]:
test = test_df.merge(test_last[used_columns],how = 'left',on = 'card_id')

In [12]:
del train['outliers'],test['outliers']
gc.collect()

116

In [13]:
print(train.shape,test.shape)

(201917, 479) (123623, 478)


In [14]:
#del train_df,test_df,train_last,test_last
#gc.collect()

In [15]:
"""def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in tqdm(list(range(df.shape[1]))):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)
duplicateColumnNames = getDuplicateColumns(train)"""

"def getDuplicateColumns(df):\n    '''\n    Get a list of duplicate columns.\n    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.\n    :param df: Dataframe object\n    :return: List of columns whose contents are duplicates.\n    '''\n    duplicateColumnNames = set()\n    # Iterate over all the columns in dataframe\n    for x in tqdm(list(range(df.shape[1]))):\n        # Select column at xth index.\n        col = df.iloc[:, x]\n        # Iterate over all the columns in DataFrame from (x+1)th index till end\n        for y in range(x + 1, df.shape[1]):\n            # Select column at yth index.\n            otherCol = df.iloc[:, y]\n            # Check if two columns at x 7 y index are equal\n            if col.equals(otherCol):\n                duplicateColumnNames.add(df.columns.values[y])\n \n    return list(duplicateColumnNames)\nduplicateColumnNames = getDuplicateColumns(train)"

In [16]:
duplicateColumnNames = ['h_month_lag_max', 'n_month_lag_min', 'new_card_id_count', 'new_transactions_count', 'n_af_sum', 'new_purchase_month_min', 'n_af_mean', 'h_max_min_diff', 'n_af_count', 'hist_card_id_count', 'n_how_many_unique_subsector_id', 'card_id_cnt_total', 'n_max_min_diff', 'installments_count_std', 'new_purchase_month_max', 'installments_count_mean', 'n_ins_count', 'n_af_max', 'a_transactions_count', 'n_how_many_merchant_category_id_visited', 'h_how_many_merchant_category_id_visited', 'n_af_median', 'h_ins_count', 'h_how_many_unique_subsector_id', 'h_how_many_merchant_id_visited', 'n_how_many_merchant_id_visited', 'h_month_lag_min', 'h_af_count', 'n_month_lag_max']

In [17]:
print(duplicateColumnNames)

['h_month_lag_max', 'n_month_lag_min', 'new_card_id_count', 'new_transactions_count', 'n_af_sum', 'new_purchase_month_min', 'n_af_mean', 'h_max_min_diff', 'n_af_count', 'hist_card_id_count', 'n_how_many_unique_subsector_id', 'card_id_cnt_total', 'n_max_min_diff', 'installments_count_std', 'new_purchase_month_max', 'installments_count_mean', 'n_ins_count', 'n_af_max', 'a_transactions_count', 'n_how_many_merchant_category_id_visited', 'h_how_many_merchant_category_id_visited', 'n_af_median', 'h_ins_count', 'h_how_many_unique_subsector_id', 'h_how_many_merchant_id_visited', 'n_how_many_merchant_id_visited', 'h_month_lag_min', 'h_af_count', 'n_month_lag_max']


In [18]:
train.drop(duplicateColumnNames,axis=1,inplace=True)
test.drop(duplicateColumnNames,axis=1,inplace=True)

In [19]:
print(train.shape,test.shape)

(201917, 450) (123623, 449)


In [20]:
train.head()

,card_id,feature_1,feature_2,feature_3,first_active_month,target,quarter,elapsed_time,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_mean,feature_max,feature_min,feature_var,hist_subsector_id_nunique,hist_merchant_id_nunique,hist_merchant_category_id_nunique,hist_month_nunique,hist_month_mean,hist_month_min,hist_month_max,hist_hour_nunique,hist_hour_mean,hist_hour_min,hist_hour_max,hist_weekofyear_nunique,hist_weekofyear_mean,hist_weekofyear_min,hist_weekofyear_max,hist_weekday_mean,hist_day_nunique,hist_day_mean,hist_day_min,hist_purchase_amount_sum,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_skew,hist_installments_sum,hist_installments_max,hist_installments_mean,hist_installments_var,hist_installments_skew,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_max,hist_month_lag_min,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_skew,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_authorized_flag_mean,hist_weekend_mean,hist_category_1_mean,hist_category_2_mean,hist_category_3_mean,hist_card_id_size,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_3_mean_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_subsector_id_nunique,new_merchant_id_nunique,new_merchant_category_id_nunique,new_month_mean,new_month_min,new_month_max,new_hour_nunique,new_hour_mean,new_hour_min,new_hour_max,new_weekofyear_nunique,new_weekofyear_mean,new_weekofyear_min,new_weekofyear_max,new_weekday_mean,new_weekday_min,new_weekday_max,new_day_nunique,new_day_mean,new_day_min,new_day_max,new_purchase_amount_sum,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_skew,new_installments_sum,new_installments_max,new_installments_mean,new_installments_var,new_installments_skew,new_purchase_date_max,new_purchase_date_min,new_month_lag_max,new_month_lag_min,new_month_lag_mean,new_month_lag_var,new_month_lag_skew,new_month_diff_mean,new_month_diff_var,new_month_diff_skew,new_weekend_mean,new_category_1_mean,new_category_2_mean,new_category_3_mean,new_card_id_size,new_price_mean,new_price_max,new_price_min,new_price_var,new_Christmas_Day_2017_mean,new_Children_day_2017_mean,new_Black_Friday_2017_mean,new_Mothers_Day_2018_mean,new_duration_mean,new_duration_min,new_duration_max,new_duration_var,new_duration_skew,new_amount_month_ratio_mean,new_amount_month_ratio_min,new_amount_month_ratio_max,new_amount_month_ratio_var,new_amount_month_ratio_skew,new_category_2_mean_mean,new_category_3_mean_mean,new_purchase_date_diff,new_purchase_date_average,new_purchase_date_uptonow,new_purchase_date_uptomin,hist_first_buy,hist_last_buy,new_first_buy,new_last_buy,card_id_total,card_id_cnt_ratio,purchase_amount_total,purchase_amount_mean,purchase_amount_max,purchase_amount_min,purchase_amount_ratio,month_diff_mean,month_diff_ratio,month_lag_mean,month_lag_max,month_lag_min,category_1_mean,installments_total,installments_mean,installments_max,installments_ratio,price_total,price_mean,price_max,duration_mean,duration_min,duration_max,amount_month_ratio_mean,amount_month_ratio_min,amount_month_ratio_max,new_CLV,hist_CLV,CLV_ratio,active_year,active_month,h_af_sum,h_af_min,h_af_max,h_af_std,h_af_mean,h_af_median,h_how_many_city_visited,h_most_visited_

In [26]:
FEATS_EXCLUDED = ['hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size']

In [28]:
train.drop(FEATS_EXCLUDED,axis=1,inplace=True)
test.drop(FEATS_EXCLUDED,axis=1,inplace=True)

In [29]:
print(train.shape,test.shape)

(201917, 444) (123623, 443)


In [30]:
train.to_csv('train_final.csv',index=False)
test.to_csv('test_final.csv',index=False)

In [ ]:
y = train['target']
del train['target']

In [ ]:
stratified = False

if stratified:
    folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=SEED)
else:
    folds = KFold(n_splits= 5, shuffle=True, random_state=SEED)

# Create arrays and dataframes to store results
oof_preds = np.zeros(train.shape[0])
sub_preds = np.zeros(test.shape[0])
feature_importance_df = pd.DataFrame()

In [ ]:
temp = train.dtypes.reset_index(drop=False)
temp.columns = ['column','type']
for i in [x for x in list(temp[temp['type'] == 'object']['column']) if x not in ['card_id', 'first_active_month']]:
    print(i)
    le = preprocessing.LabelEncoder()
    le.fit(list(train[i].values) + list(test[i].values ) )
    train[i] = le.transform( list(train[i].values) )
    test[i] = le.transform( list(test[i].values) )
feats = [c for c in train.columns if c not in ['card_id', 'first_active_month'] + FEATS_EXCLUDED]

In [ ]:
#categorical_features = ['h_most_common_category_1', 'h_second_most_common_category_1', 'h_most_common_category_3', 'h_second_most_common_category_3', 'h_third_most_common_category_3', 'h_most_common_merchant_id', 'h_second_most_common_merchant_id', 'h_third_most_common_merchant_id', 'n_most_common_category_1', 'n_second_most_common_category_1', 'n_most_common_category_3', 'n_second_most_common_category_3', 'n_third_most_common_category_3', 'n_most_common_merchant_id', 'n_second_most_common_merchant_id', 'n_third_most_common_merchant_id']

In [ ]:
# rmse
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# k-fold
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
    train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
    valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

    # set data structure
    lgb_train = lgb.Dataset(train_x,
                            label=train_y,
                            free_raw_data=False)
    lgb_test = lgb.Dataset(valid_x,
                           label=valid_y,
                           free_raw_data=False)

    # params optimized by optuna
    params = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'n_jobs':-1,
         'learning_rate': 0.02,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "bagging_seed": SEED,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": SEED,
         "verbosity": -1}

    reg = lgb.train(
                    params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_test],
                    valid_names=['train', 'test'],
                    num_boost_round=10000,
                    early_stopping_rounds= 200,
                    verbose_eval=200
                    )

    oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
    sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
    del reg, train_x, train_y, valid_x, valid_y
    gc.collect()
print("Average RMSE = {}".format(rmse(y,oof_preds)))

In [ ]:
"""
def get_feature_importances(data, shuffle):
    # Gather real features
    #train_features = [f for f in data if f not in ['target', 'object_id']]
    # Go over fold and keep track of CV score (train and valid) and feature importances
    
    # Shuffle target if required
    y_ = y.copy()

    if shuffle:
        y_ = y_.copy().sample(frac=1.0)
        y_ = pd.Series(y_)
    
    dtrain = lgb.Dataset(data[feats], y_, free_raw_data=False, silent=True)
    
    # Fit LightGBM in RF mode, yes it's quicker than sklearn RandomForest
    param = {'num_leaves': 111,
             'min_data_in_leaf': 149, 
             'objective':'regression',
             'max_depth': 9,
             'learning_rate': 0.02,
             "boosting": "gbdt",
             "feature_fraction": 0.7522,
             "bagging_freq": 1,
             "bagging_fraction": 0.7083 ,
             "bagging_seed": SEED,
             "metric": 'rmse',
             "lambda_l1": 0.2634,
             "random_state": 51,
             "verbosity": -1}
    clf = lgb.train(params=param, train_set=dtrain, num_boost_round=200)

    # Get feature importances
    imp_df = pd.DataFrame()
    imp_df["feature"] = list(feats)
    imp_df["importance_gain"] = clf.feature_importance(importance_type='gain')
    imp_df["importance_split"] = clf.feature_importance(importance_type='split')
    #imp_df['trn_score'] = roc_auc_score(pd.get_dummies(y), clf.predict_proba(data[train_features]))
    
    return imp_df
"""

In [ ]:
"""actual_imp_df = get_feature_importances(data=train, shuffle=False)"""

In [ ]:
"""
null_imp_df = pd.DataFrame()
nb_runs = 80
import time
start = time.time()
dsp = ''
for i in range(nb_runs):
    # Get current run importances
    imp_df = get_feature_importances(data=train, shuffle=True)
    imp_df['run'] = i + 1 
    # Concat the latest importances with the old ones
    null_imp_df = pd.concat([null_imp_df, imp_df], axis=0)
    # Erase previous message
    for l in range(len(dsp)):
        print('\b', end='', flush=True)
    # Display current run and time used
    spent = (time.time() - start) / 60
    dsp = 'Done with %4d of %4d (Spent %5.1f min)' % (i + 1, nb_runs, spent)
    print(dsp, end='', flush=True)
"""

In [ ]:
"""
temp = null_imp_df.groupby('feature')['importance_gain','importance_split'].agg(['mean','std']).reset_index(drop=False)
temp.columns = ['feature','gain_mean','gain_std','split_mean','split_std']
temp['gain_2_sigma'] = temp['gain_mean'] + temp['gain_std']*1
temp['split_2_sigma'] = temp['split_mean'] + temp['split_std']*1
temp2 = actual_imp_df.merge(temp[['feature','gain_2_sigma','split_2_sigma']],how='left',on = 'feature')
#temp2[(temp2['importance_gain'] > temp2['gain_2_sigma']) & (temp2['importance_split'] > temp2['split_2_sigma'] )].head()
imp_columns = list(temp2[(temp2['importance_gain'] > temp2['gain_2_sigma']) \
      & (temp2['importance_split'] > temp2['split_2_sigma'] ) ]['feature'])
"""

In [ ]:
imp_columns = ['feature_1', 'elapsed_time', 'days_feature1', 'days_feature2', 'days_feature3', 'feature_sum', 'feature_mean', 'feature_min', 'hist_subsector_id_nunique', 'hist_merchant_id_nunique', 'hist_merchant_category_id_nunique', 'hist_month_nunique', 'hist_month_max', 'hist_weekofyear_nunique', 'hist_weekofyear_min', 'hist_weekofyear_max', 'hist_day_nunique', 'hist_installments_sum', 'hist_month_lag_min', 'hist_month_lag_mean', 'hist_month_lag_var', 'hist_month_diff_max', 'hist_month_diff_min', 'hist_month_diff_mean', 'hist_authorized_flag_mean', 'hist_category_1_mean', 'hist_price_sum', 'hist_price_var', 'hist_Mothers_Day_2017_mean', 'hist_fathers_day_2017_mean', 'hist_Children_day_2017_mean', 'hist_Valentine_Day_2017_mean', 'hist_duration_mean', 'hist_duration_min', 'hist_duration_max', 'hist_amount_month_ratio_min', 'hist_purchase_date_diff', 'hist_purchase_date_average', 'hist_purchase_date_uptonow', 'hist_purchase_date_uptomin', 'new_merchant_id_nunique', 'new_month_mean', 'new_month_max', 'new_weekofyear_mean', 'new_weekofyear_min', 'new_weekofyear_max', 'new_day_nunique', 'new_day_mean', 'new_day_min', 'new_day_max', 'new_purchase_amount_sum', 'new_purchase_amount_max', 'new_installments_mean', 'new_month_lag_max', 'new_month_lag_min', 'new_month_lag_mean', 'new_category_1_mean', 'new_Christmas_Day_2017_mean', 'new_Children_day_2017_mean', 'new_Black_Friday_2017_mean', 'new_Mothers_Day_2018_mean', 'new_duration_mean', 'new_duration_min', 'new_duration_max', 'new_amount_month_ratio_max', 'new_category_3_mean_mean', 'new_purchase_date_diff', 'new_purchase_date_average', 'new_purchase_date_uptonow', 'new_purchase_date_uptomin', 'hist_last_buy', 'card_id_cnt_ratio', 'purchase_amount_total', 'month_diff_mean', 'month_lag_max', 'month_lag_min', 'category_1_mean', 'installments_total', 'installments_ratio', 'price_total', 'duration_min', 'new_CLV', 'active_year', 'h_af_std', 'h_af_mean', 'h_most_common_category_1', 'h_num_second_most_common_category_1', 'h_ins_sum', 'h_ins_mean', 'h_num_most_common_merchant_id', 'h_num_second_most_common_merchant_id', 'h_binned_purchase_amount_sum', 'h_binned_purchase_amount_min', 'h_binned_purchase_amount_mean', 'h_month_lag_sum', 'h_month_lag_mean', 'h_purchase_amount_sum', 'h_purchase_amount_mean', 'h_purchase_amount_median', 'h_max_25_percentile_diff', 'h_75_percentile_min_diff', 'h_50_percentile_min_diff', 'h_most_common_subsector_id', 'n_num_most_common_category_1', 'n_ins_min', 'n_binned_purchase_amount_sum', 'n_month_lag_mean', 'n_purchase_amount_sum', 'n_purchase_amount_max', 'n_purchase_amount_std', 'n_purchase_amount_mean', 'n_max_25_percentile_diff', 'h_transactions_count', 'h_category_1_sum', 'h_category_1_mean', 'h_month_diff_mean', 'a_category_1_sum', 'a_category_1_mean', 'a_category_3_B_mean', 'a_purchase_month_max', 'a_purchase_month_std', 'a_month_diff_mean', 'new_category_1_sum', 'new_category_3_B_mean', 'new_purchase_month_mean', 'new_purchase_month_std', 'month_lag_std', 'purchase_amount_count_mean', 'purchase_amount_mean_mean', 'purchase_amount_min_mean', 'purchase_amount_max_mean', 'purchase_amount_max_std', 'installments_sum_mean', 'installments_sum_std', 'installments_max_mean', 'category_1_purchase_amount_mean', 'category_1_purchase_amount_max', 'installments_purchase_amount_mean', 'installments_purchase_amount_max', 'city_id_purchase_amount_mean', 'city_id_purchase_amount_max']

In [ ]:
only_in_last = [x for x in imp_columns if x not in used_features]

In [ ]:
train = train_df.merge(train_last[['card_id'] + only_in_last],how = 'left',on = 'card_id')

In [ ]:
test = test_df.merge(test_last[['card_id'] + only_in_last],how = 'left',on = 'card_id')

In [ ]:
print(train.shape,test.shape)

In [ ]:
y = train['target']
del train['target']

In [ ]:
train.head()

In [ ]:
imp_columns = [x for x in train.columns if x != 'card_id']

In [ ]:
temp = train.dtypes.reset_index(drop=False)
temp.columns = ['column','type']
for i in [x for x in list(temp[temp['type'] == 'object']['column']) if x not in ['card_id', 'first_active_month']]:
    print(i)
    le = preprocessing.LabelEncoder()
    le.fit(list(train[i].values) + list(test[i].values ) )
    train[i] = le.transform( list(train[i].values) )
    test[i] = le.transform( list(test[i].values) )
feats = [c for c in train.columns if c not in ['card_id', 'first_active_month'] + FEATS_EXCLUDED]

In [ ]:
# k-fold
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train[feats],y)):
    train_x, train_y = train[feats].iloc[train_idx], y.iloc[train_idx]
    valid_x, valid_y = train[feats].iloc[valid_idx], y.iloc[valid_idx]

    # set data structure
    lgb_train = lgb.Dataset(train_x,
                            label=train_y,
                            free_raw_data=False)
    lgb_test = lgb.Dataset(valid_x,
                           label=valid_y,
                           free_raw_data=False)

    # params optimized by optuna
    params = {'num_leaves': 111,
         'min_data_in_leaf': 149, 
         'objective':'regression',
         'max_depth': 9,
         'learning_rate': 0.02,
         "boosting": "gbdt",
         "feature_fraction": 0.7522,
         "bagging_freq": 1,
         "bagging_fraction": 0.7083 ,
         "bagging_seed": SEED,
         "metric": 'rmse',
         "lambda_l1": 0.2634,
         "random_state": SEED,
         "verbosity": -1}

    reg = lgb.train(
                    params,
                    lgb_train,
                    valid_sets=[lgb_train, lgb_test],
                    valid_names=['train', 'test'],
                    num_boost_round=10000,
                    early_stopping_rounds= 200,
                    verbose_eval=200
                    )

    oof_preds[valid_idx] = reg.predict(valid_x, num_iteration=reg.best_iteration)
    sub_preds += reg.predict(test[feats], num_iteration=reg.best_iteration) / folds.n_splits

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = np.log1p(reg.feature_importance(importance_type='gain', iteration=reg.best_iteration))
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
    del reg, train_x, train_y, valid_x, valid_y
    gc.collect()
print("Average RMSE = {}".format(rmse(y,oof_preds)))

In [ ]:
test['target'] = sub_preds

In [ ]:
submission_file_name = '../my_submissions/deneme2'
test[['card_id', 'target']].to_csv(submission_file_name, index=False)